In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.colors import LinearSegmentedColormap
import ipywidgets as widgets
from IPython.display import display, clear_output
import json
import warnings
warnings.filterwarnings('ignore')

# Загрузка данных
try:
    # Пробуем загрузить CSV
    df = pd.read_csv('elden_ring_characters.csv')
    print("Данные успешно загружены из CSV")
except:
    try:
        # Если CSV не найден, пробуем загрузить JSON
        with open('elden_ring_characters.json', 'r', encoding='utf-8') as f:
            data = json.load(f)
        df = pd.DataFrame(data)
        print("Данные успешно загружены из JSON")
    except:
        print("Ошибка загрузки данных! Убедитесь, что файлы elden_ring_characters.csv или elden_ring_characters.json находятся в текущей директории.")
        # Создаем тестовые данные, если не удалось загрузить
        df = pd.DataFrame({
            'name': ['Тестовый персонаж 1', 'Тестовый персонаж 2', 'Тестовый персонаж 3'],
            'health': [1000, 2000, 3000],
            'is_boss': [True, False, False],
            'is_miniboss': [False, True, False],
            'is_npc': [False, False, True],
            'has_quest': [False, False, True],
            'is_hostile': [True, True, False],
            'is_friendly': [False, False, True],
            'faction': ['Unknown', 'Unknown', 'Unknown'],
            'location': ['Unknown', 'Unknown', 'Unknown'],
            'role': ['Enemy', 'Enemy', 'NPC']
        })

# Подготовка данных
df['character_type'] = 'Enemy'  # Базовый тип
df.loc[df['is_boss'], 'character_type'] = 'Boss'
df.loc[df['is_miniboss'] & ~df['is_boss'], 'character_type'] = 'Mini-Boss'
df.loc[df['is_npc'] & ~df['is_boss'] & ~df['is_miniboss'], 'character_type'] = 'NPC'

# Преобразуем текстовые поля в категориальные, если они не пустые
for col in ['faction', 'location', 'role']:
    non_empty_mask = (df[col] != 'Unknown') & (df[col].notna())
    if non_empty_mask.any():
        df[col] = df[col].astype(str)

# Создаем кастомную цветовую схему Elden Ring
elden_ring_colors = {
    'gold': '#D4AF37',
    'crimson': '#DC143C',
    'dark_gold': '#B8860B',
    'forest_green': '#228B22',
    'teal': '#008080',
    'purple': '#800080',
    'dark_slate': '#2F4F4F',
    'goldenrod': '#DAA520'
}

# Создаем цветовые карты для разных визуализаций
type_colors = {
    'Boss': elden_ring_colors['crimson'],
    'Mini-Boss': elden_ring_colors['dark_gold'],
    'NPC': elden_ring_colors['forest_green'],
    'Enemy': elden_ring_colors['dark_slate']
}

# Создаем специальную цветовую палитру Elden Ring
elden_ring_palette = sns.color_palette([
    elden_ring_colors['gold'],
    elden_ring_colors['crimson'],
    elden_ring_colors['dark_gold'],
    elden_ring_colors['forest_green'],
    elden_ring_colors['teal'],
    elden_ring_colors['purple'],
    elden_ring_colors['dark_slate'],
    elden_ring_colors['goldenrod']
])

# Функция для получения цвета персонажа
def get_character_color(char_type):
    return type_colors.get(char_type, elden_ring_colors['dark_slate'])

# Функции для создания различных визуализаций
def plot_health_comparison(selected_chars, title_size=16, axis_label_size=12):
    """Создает столбчатую диаграмму для сравнения здоровья персонажей"""
    if not selected_chars:
        plt.figure(figsize=(10, 6))
        plt.text(0.5, 0.5, "Выберите персонажей для сравнения", 
                 horizontalalignment='center', verticalalignment='center', fontsize=14)
        plt.axis('off')
        plt.show()
        return
        
    selected_df = df[df['name'].isin(selected_chars)].sort_values('health', ascending=False)
    
    plt.figure(figsize=(12, 8))
    
    colors = [get_character_color(char_type) for char_type in selected_df['character_type']]
    
    ax = sns.barplot(x='name', y='health', data=selected_df, palette=colors)
    
    plt.title('Сравнение здоровья персонажей', fontsize=title_size)
    plt.xlabel('Персонаж', fontsize=axis_label_size)
    plt.ylabel('Здоровье (HP)', fontsize=axis_label_size)
    plt.xticks(rotation=45, ha='right')
    
    # Добавляем значения над столбцами
    for i, p in enumerate(ax.patches):
        ax.annotate(f"{int(p.get_height()):,}",
                    (p.get_x() + p.get_width() / 2., p.get_height()),
                    ha='center', va='bottom',
                    fontsize=10, color='black',
                    xytext=(0, 5),
                    textcoords='offset points')
    
    # Добавляем легенду для типов персонажей
    handles = [plt.Rectangle((0,0),1,1, color=color) for color in type_colors.values()]
    labels = list(type_colors.keys())
    plt.legend(handles, labels, title="Тип персонажа", loc='upper right')
    
    plt.tight_layout()
    plt.show()

def plot_character_types(selected_chars, title_size=16):
    """Создает круговую диаграмму типов персонажей"""
    if not selected_chars:
        plt.figure(figsize=(10, 6))
        plt.text(0.5, 0.5, "Выберите персонажей для анализа", 
                 horizontalalignment='center', verticalalignment='center', fontsize=14)
        plt.axis('off')
        plt.show()
        return
        
    selected_df = df[df['name'].isin(selected_chars)]
    
    type_counts = selected_df['character_type'].value_counts()
    
    plt.figure(figsize=(10, 8))
    
    colors = [get_character_color(char_type) for char_type in type_counts.index]
    
    wedges, texts, autotexts = plt.pie(
        type_counts,
        labels=type_counts.index,
        autopct='%1.1f%%',
        colors=colors,
        startangle=90,
        shadow=True,
        wedgeprops={'edgecolor': 'black', 'linewidth': 1}
    )
    
    # Настраиваем внешний вид
    for text in texts:
        text.set_fontsize(12)
    for autotext in autotexts:
        autotext.set_fontsize(11)
        autotext.set_color('white')
    
    plt.title('Распределение типов персонажей', fontsize=title_size)
    plt.axis('equal')  # Чтобы круг был круглым
    
    plt.tight_layout()
    plt.show()

def plot_npc_analysis(selected_chars=None, title_size=16):
    """Анализ NPC персонажей: их характеристики и квесты"""
    # Если не указаны конкретные персонажи, берем всех NPC
    if not selected_chars:
        npc_df = df[df['character_type'] == 'NPC']
    else:
        # Иначе берем NPC из выбранных персонажей
        npc_df = df[df['name'].isin(selected_chars) & (df['character_type'] == 'NPC')]
    
    if npc_df.empty:
        plt.figure(figsize=(10, 6))
        plt.text(0.5, 0.5, "Нет NPC персонажей в выбранных данных", 
                 horizontalalignment='center', verticalalignment='center', fontsize=14)
        plt.axis('off')
        plt.show()
        return
    
    # Ограничиваем количество NPC для отображения (не больше 15)
    if len(npc_df) > 15:
        npc_df = npc_df.sample(15)
    
    # Создаем данные для визуализации свойств NPC
    fig, axes = plt.subplots(1, 2, figsize=(18, 8))
    
    # 1. Сравнение характеристик NPC (квесты, дружелюбие)
    # Подготавливаем данные для первого графика
    char_data = []
    for idx, row in npc_df.iterrows():
        char_data.append({
            'name': row['name'],
            'has_quest': 1 if row['has_quest'] else 0,
            'is_friendly': 1 if row['is_friendly'] else 0,
            'is_hostile': 1 if row['is_hostile'] else 0
        })
    
    char_df = pd.DataFrame(char_data)
    
    # Преобразуем данные для группированной столбчатой диаграммы
    char_melted = pd.melt(char_df, id_vars=['name'], 
                         value_vars=['has_quest', 'is_friendly', 'is_hostile'],
                         var_name='characteristic', value_name='value')
    
    # Переименовываем характеристики для лучшей читаемости
    char_melted['characteristic'] = char_melted['characteristic'].replace({
        'has_quest': 'Имеет квест',
        'is_friendly': 'Дружелюбный',
        'is_hostile': 'Враждебный'
    })
    
    # Рисуем группированную столбчатую диаграмму
    sns.barplot(x='name', y='value', hue='characteristic', data=char_melted, ax=axes[0])
    axes[0].set_title('Характеристики NPC персонажей', fontsize=title_size)
    axes[0].set_xlabel('Персонаж', fontsize=12)
    axes[0].set_ylabel('Значение', fontsize=12)
    axes[0].tick_params(axis='x', rotation=45)
    axes[0].set_ylim(0, 1.2)  # Ограничиваем ось Y для бинарных значений
    
    # 2. Распределение локаций NPC
    # Подготавливаем данные о локациях
    location_data = []
    for idx, row in npc_df.iterrows():
        locations = str(row['location']).split(', ')
        for loc in locations:
            if loc != 'Unknown':
                location_data.append(loc)
    
    location_counts = pd.Series(location_data).value_counts()
    
    # Если слишком много локаций, группируем редкие
    if len(location_counts) > 8:
        other_count = location_counts[8:].sum()
        location_counts = location_counts[:8]
        if other_count > 0:
            location_counts['Другие'] = other_count
    
    # Рисуем круговую диаграмму для локаций
    if not location_counts.empty:
        wedges, texts, autotexts = axes[1].pie(
            location_counts, 
            labels=location_counts.index,
            autopct='%1.1f%%',
            colors=elden_ring_palette[:len(location_counts)],
            startangle=90,
            shadow=True
        )
        
        # Настраиваем внешний вид
        for text in texts:
            text.set_fontsize(10)
        for autotext in autotexts:
            autotext.set_fontsize(9)
            autotext.set_color('white')
            
        axes[1].set_title('Распределение NPC по локациям', fontsize=title_size)
        axes[1].axis('equal')
    else:
        axes[1].text(0.5, 0.5, "Нет данных о локациях NPC", 
                    horizontalalignment='center', verticalalignment='center', fontsize=14)
        axes[1].axis('off')
    
    plt.tight_layout()
    plt.show()

def plot_location_heatmap(selected_chars, title_size=16):
    """Создает тепловую карту локаций персонажей"""
    if not selected_chars:
        plt.figure(figsize=(10, 6))
        plt.text(0.5, 0.5, "Выберите персонажей для анализа", 
                 horizontalalignment='center', verticalalignment='center', fontsize=14)
        plt.axis('off')
        plt.show()
        return
        
    selected_df = df[df['name'].isin(selected_chars)]
    
    # Если нет данных о локациях или все локации 'Unknown'
    if selected_df['location'].isin(['Unknown']).all():
        plt.figure(figsize=(10, 6))
        plt.text(0.5, 0.5, "Нет информации о локациях для выбранных персонажей", 
                 horizontalalignment='center', verticalalignment='center', fontsize=14)
        plt.axis('off')
        plt.show()
        return
    
    # Разбиваем множественные значения локаций
    location_lists = selected_df['location'].str.split(', ').apply(
        lambda x: [item.strip() for item in x] if isinstance(x, list) else [str(x).strip()]
    )
    
    # Создаем матрицу локаций и персонажей
    locations = set()
    for locs in location_lists:
        locations.update(locs)
    
    # Отбрасываем Unknown локацию, если есть и другие
    if 'Unknown' in locations and len(locations) > 1:
        locations.remove('Unknown')
    
    if not locations:
        plt.figure(figsize=(10, 6))
        plt.text(0.5, 0.5, "Нет информации о локациях для выбранных персонажей", 
                 horizontalalignment='center', verticalalignment='center', fontsize=14)
        plt.axis('off')
        plt.show()
        return
    
    locations = sorted(list(locations))
    characters = selected_df['name'].tolist()
    
    location_matrix = np.zeros((len(characters), len(locations)))
    
    for i, char in enumerate(characters):
        # Используем .loc вместо .iloc для поиска по значению
        mask = selected_df['name'] == char
        if mask.any():  # Проверяем, есть ли совпадения
            char_locations = location_lists[mask].iloc[0]
        else:
            char_locations = []  # Если персонаж не найден, используем пустой список
        
        for loc in char_locations:
            if loc in locations:  # пропускаем 'Unknown'
                j = locations.index(loc)
                location_matrix[i, j] = 1
    
    # Если слишком много локаций, объединяем редкие
    MAX_LOCATIONS = 15
    if len(locations) > MAX_LOCATIONS:
        # Находим наиболее частые локации
        location_counts = location_matrix.sum(axis=0)
        top_indices = np.argsort(location_counts)[-MAX_LOCATIONS+1:]
        
        # Создаем новую матрицу
        new_matrix = np.zeros((len(characters), MAX_LOCATIONS))
        new_locations = [locations[i] for i in top_indices]
        
        # Копируем данные для топ-локаций
        for i in range(len(top_indices)):
            new_matrix[:, i] = location_matrix[:, top_indices[i]]
        
        # Объединяем остальные локации в "Другие"
        other_indices = [i for i in range(len(locations)) if i not in top_indices]
        if other_indices:
            other_column = np.zeros(len(characters))
            for i in other_indices:
                other_column = np.logical_or(other_column, location_matrix[:, i])
            new_matrix[:, -1] = other_column
            new_locations.append('Другие')
        
        location_matrix = new_matrix
        locations = new_locations
    
    plt.figure(figsize=(14, 10))
    
    # Создаем кастомную цветовую карту для тепловой карты
    elden_cmap = LinearSegmentedColormap.from_list(
        "elden_ring_map", [
            "#2F4F4F",  # темно-серый
            elden_ring_colors['gold']  # золотой
        ]
    )
    
    # Настраиваем параметры для heatmap
    sns.heatmap(
        location_matrix,
        cmap=elden_cmap,
        linewidths=1,
        linecolor='white',
        cbar=False,
        xticklabels=locations,
        yticklabels=characters
    )
    
    plt.title('Карта локаций персонажей', fontsize=title_size)
    plt.xlabel('Локации', fontsize=12)
    plt.ylabel('Персонажи', fontsize=12)
    plt.xticks(rotation=45, ha='right')
    plt.yticks(rotation=0)
    
    plt.tight_layout()
    plt.show()

def plot_radar_chart(selected_chars, title_size=16):
    """Создает лепестковую диаграмму характеристик персонажей"""
    if not selected_chars:
        plt.figure(figsize=(10, 6))
        plt.text(0.5, 0.5, "Выберите персонажей для анализа", 
                 horizontalalignment='center', verticalalignment='center', fontsize=14)
        plt.axis('off')
        plt.show()
        return
        
    selected_df = df[df['name'].isin(selected_chars)]
    
    # Ограничиваем количество персонажей для radar chart (слишком много делает график нечитаемым)
    if len(selected_df) > 8:
        selected_df = selected_df.iloc[:8]
        print(f"⚠️ Отображаются только первые 8 персонажей для читаемости графика")
    
    # Определяем категории для radar chart
    categories = ['health', 'is_boss', 'is_miniboss', 'is_npc', 'has_quest', 'is_hostile', 'is_friendly']
    display_categories = ['Здоровье', 'Босс', 'Мини-босс', 'NPC', 'Имеет квест', 'Враждебный', 'Дружелюбный']
    
    # Количество категорий
    N = len(categories)
    
    # Нормализуем значения для сравнения
    radar_df = selected_df.copy()
    max_health = radar_df['health'].max()
    if max_health > 0:
        radar_df['health'] = radar_df['health'] / max_health
    
    # Рассчитываем углы для каждой категории
    angles = [n / float(N) * 2 * np.pi for n in range(N)]
    angles += angles[:1]  # Замыкаем круг
    
    # Создаем лепестковую диаграмму
    fig, ax = plt.subplots(figsize=(10, 10), subplot_kw=dict(polar=True))
    
    # Добавляем линии для каждого персонажа
    for i, (idx, row) in enumerate(radar_df.iterrows()):
        values = [
            row['health'],
            float(row['is_boss']),
            float(row['is_miniboss']),
            float(row['is_npc']),
            float(row['has_quest']),
            float(row['is_hostile']),
            float(row['is_friendly'])
        ]
        values += values[:1]  # Замыкаем значения
        
        # Определяем цвет по типу персонажа
        color = get_character_color(row['character_type'])
        
        # Рисуем линию и заполняем область
        ax.plot(angles, values, linewidth=2, linestyle='solid', color=color, label=row['name'])
        ax.fill(angles, values, color=color, alpha=0.25)
    
    # Устанавливаем метки категорий
    ax.set_xticks(angles[:-1])
    ax.set_xticklabels(display_categories)
    
    # Настраиваем внешний вид
    ax.set_yticklabels([])  # Убираем метки по оси Y
    ax.grid(True, linestyle='--', alpha=0.7)
    
    plt.title('Характеристики персонажей', fontsize=title_size, y=1.1)
    plt.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))
    
    plt.tight_layout()
    plt.show()

# Создаем интерактивный дашборд без фильтрации по фракциям
def create_dashboard():
    """Создает интерактивный дашборд с использованием ipywidgets"""
    # Объявляем некоторые переменные как глобальные для обработчиков событий
    global character_selector, selected_characters
    # Сохраняем выбранные персонажи для обеспечения постоянства выбора
    selected_characters = []
    # Виджеты для выбора персонажей
    character_selector = widgets.SelectMultiple(
        options=sorted(df['name'].tolist()),
        value=[],
        description='Персонажи:',
        disabled=False,
        layout=widgets.Layout(width='90%', height='200px')
    )
    # Инициализируем выбор первыми 5 персонажами
    if len(df) > 0:
        character_selector.value = sorted(df['name'].tolist())[:5]
        selected_characters = sorted(df['name'].tolist())[:5]
    # Фильтры по типам персонажей
    type_filter = widgets.SelectMultiple(
        options=sorted(df['character_type'].unique().tolist()),
        value=sorted(df['character_type'].unique().tolist()),
        description='Тип:',
        disabled=False,
        layout=widgets.Layout(width='45%', height='100px')
    )
    # Выбор типа визуализации
    chart_type = widgets.RadioButtons(
        options=[
            'Сравнение здоровья',
            'Типы персонажей',
            'Анализ NPC',
            'Карта локаций',
            'Характеристики (лепестковая)'
        ],
        value='Сравнение здоровья',
        description='Диаграмма:',
        disabled=False,
        layout=widgets.Layout(width='90%')
    )
    # Максимальное количество персонажей для отображения в списке
    max_chars_slider = widgets.IntSlider(
        value=20,
        min=5,
        max=100,
        step=5,
        description='Макс. персонажей в списке:',
        disabled=False,
        continuous_update=False,
        orientation='horizontal',
        readout=True,
        readout_format='d',
        layout=widgets.Layout(width='50%')
    )
    # Информационная панель для отладки
    info_output = widgets.Output(layout=widgets.Layout(width='90%', height='50px'))
    # Кнопка обновления
    update_button = widgets.Button(
        description='Обновить график',
        disabled=False,
        button_style='info',
        tooltip='Нажмите, чтобы обновить визуализацию',
        icon='refresh'
    )
    # Функция для обновления списка персонажей на основе фильтров
    def update_character_list(types, max_chars):
        global selected_characters
        with info_output:
            clear_output(wait=True)
            print(f"Применение фильтров: {len(types)} типов")
        filtered_df = df.copy()
        # Применяем фильтр по типу
        if types:
            filtered_df = filtered_df[filtered_df['character_type'].isin(types)]
        # Ограничиваем количество персонажей для отображения в списке
        filtered_df = filtered_df.head(max_chars)
        # Сохраняем текущий выбор
        current_selection = list(character_selector.value)
        # Обновляем опции выбора персонажей
        character_selector.options = sorted(filtered_df['name'].tolist())
        # Определяем, какие из выбранных персонажей все еще в списке
        valid_chars = [char for char in current_selection if char in filtered_df['name'].tolist()]
        # Если ни один из ранее выбранных персонажей не в списке, выбираем первые 5 (или все, если их меньше)
        if not valid_chars:
            if len(character_selector.options) > 0:
                new_selection = list(character_selector.options)[:min(5, len(character_selector.options))]
                character_selector.value = new_selection
                selected_characters = new_selection
            else:
                character_selector.value = []
                selected_characters = []
        else:
            # Иначе сохраняем текущий выбор
            character_selector.value = valid_chars
            selected_characters = valid_chars
        with info_output:
            print(f"Выбрано {len(character_selector.value)} из {len(character_selector.options)} персонажей")
    # Обработчики изменений
    def on_type_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            update_character_list(change['new'], max_chars_slider.value)
    def on_max_chars_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            update_character_list(type_filter.value, change['new'])
    # Когда пользователь выбирает персонажей, сохраняем выбор
    def on_character_selection_change(change):
        global selected_characters
        if change['type'] == 'change' and change['name'] == 'value':
            selected_characters = list(change['new'])
            with info_output:
                clear_output(wait=True)
                print(f"Выбрано {len(selected_characters)} персонажей")
    # Регистрируем обработчики
    type_filter.observe(on_type_change, names='value')
    max_chars_slider.observe(on_max_chars_change, names='value')
    character_selector.observe(on_character_selection_change, names='value')
    # Функция для создания визуализации
    output = widgets.Output()
    def create_visualization(button=None):
        with output:
            clear_output(wait=True)
            # Получаем текущий выбор персонажей
            chars_to_show = list(character_selector.value)
            if not chars_to_show and chart_type.value != 'Анализ NPC':
                print("Пожалуйста, выберите хотя бы одного персонажа")
                return
            # Создаем соответствующую визуализацию
            if chart_type.value == 'Сравнение здоровья':
                plot_health_comparison(chars_to_show)
            elif chart_type.value == 'Типы персонажей':
                plot_character_types(chars_to_show)
            elif chart_type.value == 'Анализ NPC':
                plot_npc_analysis(chars_to_show)
            elif chart_type.value == 'Карта локаций':
                plot_location_heatmap(chars_to_show)
            elif chart_type.value == 'Характеристики (лепестковая)':
                plot_radar_chart(chars_to_show)
    # Регистрируем обработчик нажатия кнопки
    update_button.on_click(create_visualization)
    # Обработчик изменения типа графика
    def on_chart_type_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            create_visualization()
    chart_type.observe(on_chart_type_change, names='value')
    # Создаем интерфейс дашборда
    filter_controls = widgets.HBox([type_filter])
    main_controls = widgets.VBox([
        widgets.HTML("<h2 style='text-align:center;'>Анализ персонажей Elden Ring</h2>"),
        widgets.HTML("<p style='text-align:center;'>Выберите параметры для анализа и сравнения персонажей</p>"),
        filter_controls,
        max_chars_slider,
        character_selector,
        chart_type,
        update_button,
        info_output
    ])
    dashboard = widgets.VBox([
        main_controls,
        output
    ])
    # Инициализируем дашборд
    update_character_list(type_filter.value, max_chars_slider.value)
    create_visualization()
    return dashboard

# Запускаем дашборд
dashboard = create_dashboard()
display(dashboard)

Данные успешно загружены из CSV
